In [2]:
import pandas as pd
from textblob import TextBlob

Burple =  pd.read_excel(r"ReviewsDate2016Onwards.xlsx")

import re 
bp = Burple
bp = bp.applymap(str)

ReviewList = bp['Review'].tolist()
num = len(ReviewList)

### removing emojis
RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
def strip_emoji(text):
    return RE_EMOJI.sub(r'', text)
for i in range(0, num):   #Remove emote.
    ReviewList[i] = strip_emoji(ReviewList[i])

### lower casing
RL = [x.lower() for x in ReviewList]
RL1 = RL

num = len(RL)

### Removing words witht the hashtag
spl = []
tm = ""

for i in range(0, num):
    spl = RL[i].split()
    for x in spl:
        if x.startswith('#'):
            x = ""
        else:
            x = x + " "
        tm = tm + x
    RL[i] = tm     
    tm = ""

### Removing punctuation
for i in range(0, num):   
    RL[i] = re.sub(r'[^\w\s]',' ',RL[i])

### removing the \n
for i in range(0, num):   
    RL[i] = RL[i].replace('\n','')
    
### Removing all the stop words

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

numbs = len(RL)
tests = ""
finalp = []
ww = ""

for i in range (0, numbs):
    tests = RL[i]
    
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tests)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = []                         
    comb = ""
    
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
            
    fil = len(filtered_sentence)
    
    for y in range(0, fil):
        comb = comb + " " + filtered_sentence[y]
    finalp.append(comb)
    
se = pd.Series(finalp)
bp['Cleaned_Reviews'] = se

### applying the polarity and the subjectivity

from textblob import TextBlob as tb

bp['polarity'] = bp.apply(lambda x: TextBlob(x['Cleaned_Reviews']).sentiment.polarity, axis=1)
bp['subjectivity'] = bp.apply(lambda x: TextBlob(x['Cleaned_Reviews']).sentiment.subjectivity, axis=1)

ex = pd.ExcelWriter('(Expanded) From2016_Sentimental_Burpple.xlsx', options={'encoding':'utf-8'})
bp.to_excel(ex, 'Data', index=False)

### finding average polarity

temp1 = bp.groupby('Café_Names', sort=False, as_index=False).Title.count().reset_index(drop=True)
NoR = temp1['Title'].tolist()

num1 = len(NoR)
pos = int(0)
posS = int(0)
tempList = []
totalp = float(0)
averagep = float(0)

polarity = bp['polarity'].tolist()

for i in range(0, num1):
    pos = pos + NoR[i]                            
    totalp = 0  
    averagep = 0
    for x in range(posS, pos):                     
        totalp = totalp + polarity[x]
    averagep = totalp / NoR[i]                                               
    tempList.append(averagep)                         
    posS = pos

se = pd.Series(tempList)
temp1['Average_Polarity'] = se

### finding average subjectivity

spos = int(0)
sposS = int(0)
stempList = []
totals = float(0)
averages = float(0)

subjectivity = bp['subjectivity'].tolist()

for i in range(0, num1):
    spos = spos + NoR[i]                            
    totals = 0  
    averages = 0
    for x in range(sposS, spos):                     
        totals = totals + subjectivity[x]
    averages = totals / NoR[i]                                               
    stempList.append(averages)                         
    sposS = spos

ss = pd.Series(stempList)
temp1['Average_Subjectivity'] = ss

### Putting the reviews together

temp1 = Burple.groupby('Café_Names', sort=False, as_index=False).Date.count().reset_index(drop=True)
#finding the no of rows for each unique cafe

NoR = temp1['Date'].tolist()

num1 = len(NoR)
pos = int(0)
posS = int(0)
tempList1 = []
words = ""

for i in range(0, num1):
    pos = pos + NoR[i]                             #end position
    words = ""                                     #reset words (count)
    for x in range(posS, pos):                     #Start pos, end pos
        words = words + " " + RL[x]                #concatenate
    tempList1.append(words)                         #all of the reviews bunched up tgt
    posS = pos                                     #now the the end pos is the start pos
    
### Adding the reviews to the summary dataframe (temp1)

se = pd.Series(tempList1)
temp1['Cleaned_Reviews'] = se

se = pd.Series(tempList)
temp1['Average_Polarity'] = se

ss = pd.Series(stempList)
temp1['Average_Subjectivity'] = ss

temp1

### sending to csv/ xlsx

Ave = temp1
Ave.columns = ['Café_Names', 'Count', 'Cleaned_Reviews', 'Average_Polarity', 'Average_Subjectivity']

ex = pd.ExcelWriter('(Expanded) From2016_Average_Sentimental_Burpple.xlsx', options={'encoding':'utf-8'})
Ave.to_excel(ex, 'Data', index=False)


In [ ]:
#seprating the positive/ negative/ neutral reviews

import pandas as pd
Burple =  pd.read_excel(r"(Expanded) From2016_Sentimental_Burpple.xlsx")

counta = 0
num = len(Burple)
polarity = Burple['polarity'].tolist()
neutral = []

#Nuetral Reviews
for i in range (0, num):
    if polarity[i] <0.3 and polarity[i] > (-0.3):
        neutral.append(i)

nue = Burple.iloc[neutral]

ex = pd.ExcelWriter('(Expanded) From2016_Burpple_Neutral_Reviews.xlsx', options={'encoding':'utf-8'})
nue.to_excel(ex, 'Data', index=False)

#Positive and Negative

bp = Burple.drop(Burple.index[neutral])
num = len(bp)
polarity = bp['polarity'].tolist()
positive = []
negative = []
neutral = []

for i in range (0, num):
    if polarity[i] >= 0.3:
        positive.append(i)
    if polarity[i] <= (-0.3):
        negative.append(i)
              
#for Positive reviews
bp1 = bp
pos = bp1.drop(bp1.index[negative])

ex = pd.ExcelWriter('(Expanded) From2016_Burpple_Positive_Reviews.xlsx', options={'encoding':'utf-8'})
pos.to_excel(ex, 'Data', index=False)


#for Negatove reviews
bp2 = bp
neg = bp2.drop(bp2.index[positive])

ex = pd.ExcelWriter('(Expanded) From2016_Burpple_Negative_Reviews.xlsx', options={'encoding':'utf-8'})
neg.to_excel(ex, 'Data', index=False)


In [37]:
import pandas as pd
Resturants =  pd.read_excel(r"(Expanded) From2016_Sentimental_Burpple.xlsx")
AllNames = Resturants.groupby('Café_Names', sort=False, as_index=False).Title.count().reset_index(drop=True)
NoR = AllNames['Title'].tolist()

#for Negative reviews

NegativeReviews = pd.read_excel(r"(Expanded) From2016_Burpple_Negative_Reviews.xlsx")
NegNames = NegativeReviews.groupby('Café_Names', sort=False, as_index=False).Title.count().reset_index(drop=True)
NoRNeg = NegNames['Title'].tolist()

NegReviews = []
maxNo = 0
polarity = NegativeReviews['polarity'].tolist()
Negcafename = NegativeReviews['Café_Names'].tolist()
names = AllNames['Café_Names'].tolist()
rev = NegativeReviews['Review'].tolist()
num = len(NoRNeg)
pos = 0
poss = 0

for i in names:
    if i in Negcafename:
        NegReviews.append("Yes")
    else:
        NegReviews.append("")
        
num1 = len(NegReviews)
NegReviews1 = []
NoNeg = []
counta = 0
pos1 = 0
poss1 = 0

for i in range(0, num1):
    if NegReviews[i] == "Yes":
        pos1 = pos1 + NoRNeg[counta]
        for x in range (poss1, pos1):
            if abs(polarity[x]) > maxNo:
                maxNo = abs(polarity[x])
        NegReviews1.append(rev[x])
        NoNeg.append(NoRNeg[counta])
        poss1 = pos1
        counta = counta +1
    else:
        NegReviews1.append("")
        NoNeg.append(0)
        
NegReviews2 = pd.Series(NegReviews1)

#Neutral Reviews

NeutralReviews = pd.read_excel(r"(Expanded) From2016_Burpple_Neutral_Reviews.xlsx")
NeuNames = NeutralReviews.groupby('Café_Names', sort=False, as_index=False).Title.count().reset_index(drop=True)
NoRNeu = NeuNames['Title'].tolist()

NeuReviews = []
minNo = 1
polarity = NeutralReviews['polarity'].tolist()
Neucafename = NeutralReviews['Café_Names'].tolist()
names = AllNames['Café_Names'].tolist()
rev = NeutralReviews['Review'].tolist()
num = len(NoRNeu)
pos = 0
poss = 0

for i in names:
    if i in Neucafename:
        NeuReviews.append("Yes")
    else:
        NeuReviews.append("")
        
num1 = len(NeuReviews)
NeuReviews1 = []
NoNeu = []
counta = 0
pos1 = 0
poss1 = 0

for i in range(0, num1):
    if NeuReviews[i] == "Yes":
        pos1 = pos1 + NoRNeu[counta]
        for x in range (poss1, pos1):
            if abs(polarity[x]) < minNo:
                minNo = abs(polarity[x])
        NeuReviews1.append(rev[x])
        NoNeu.append(NoRNeu[counta])
        poss1 = pos1
        counta = counta +1
    else:
        NeuReviews1.append("")
        NoNeu.append(0)
        
NeuReviews2 = pd.Series(NeuReviews1)

#Positive Reviews

PositiveReviews = pd.read_excel(r"(Expanded) From2016_Burpple_Positive_Reviews.xlsx")
PovNames = PositiveReviews.groupby('Café_Names', sort=False, as_index=False).Title.count().reset_index(drop=True)
NoRPov = PovNames['Title'].tolist()

PovReviews = []
maxNo = 1
polarity = PositiveReviews['polarity'].tolist()
Povcafename = PositiveReviews['Café_Names'].tolist()
names = AllNames['Café_Names'].tolist()
rev = PositiveReviews['Review'].tolist()
num = len(NoRPov)
pos = 0
poss = 0

for i in names:
    if i in Povcafename:
        PovReviews.append("Yes")
    else:
        PovReviews.append("")
        
num1 = len(PovReviews)
PovReviews1 = []
NoPov = []
counta = 0
pos1 = 0
poss1 = 0

for i in range(0, num1):
    if PovReviews[i] == "Yes":
        pos1 = pos1 + NoRPov[counta]
        for x in range (poss1, pos1):
            if abs(polarity[x]) > maxNo:
                maxNo = abs(polarity[x])
        PovReviews1.append(rev[x])
        NoPov.append(NoRPov[counta])
        poss1 = pos1
        counta = counta +1
    else:
        PovReviews1.append("")
        NoPov.append(0)
        
PovReviews2 = pd.Series(PovReviews1)

allsent = pd.read_excel(r"(Expanded) From2016_Average_Sentimental_Burpple.xlsx")
sent = allsent['Average_Polarity'].tolist()
sent1 = pd.Series(sent)
subj = allsent['Average_Subjectivity'].tolist()
subj1 = pd.Series(subj)

NoNeg1 = pd.Series(NoNeg)
NoNeu1 = pd.Series(NoNeu)
NoPov1 = pd.Series(NoPov)

AllNames['Average_Polarity'] = sent1.values
AllNames['Average_Subjectivity'] = subj1.values
AllNames['Negative_Review'] = NegReviews2.values
AllNames['Neutral_Review'] = NeuReviews2.values
AllNames['Positive_Review'] = PovReviews2.values
AllNames['No_of_Negative_Reviews'] = NoNeg1.values
AllNames['No_of_Neutral_Reviews'] = NoNeu1.values
AllNames['No_of_Positive_Reviews'] = NoPov1.values


AllNames.columns = ['Café_Names', 'Count','Average_Polarity', 'Average_Subjectivity','Negative_Review', 'Neutral_Review', 'Positive_Review', 'No_of_Negatives', 'No_of_Neutral', 'No_of_Positives']

ex = pd.ExcelWriter('(Expanded) From2016_Example_Reviews.xlsx', options={'encoding':'utf-8'})
AllNames.to_excel(ex, 'Data', index=False)